# Resolução de Sistemas Não Lineares


Alunos: João Guilherme Vargas, Marco Antonio Reche Rigon


Um sistema não linear é um conjunto de equações com múltiplas variáveis, onde pelo menos uma equação é não linear, ou seja, não pode ser descrito apenas com somas, subtrações e multiplicações por constantes. Não conseguimos usar métodos de resolução de sistemas lineares em sistemas não lineares. Podemos usar alguns métodos numéricos iterativos, destacando o método de Newton-Raphson

Aqui nós iremos fazer a implementação desse método para a resolução de um sistema não linear qualquer em Python. O algoritmo usará Sympy para simbolizar as equações, e numpy para fazer o tratamento com os números. Vamos precisar importar ambos para o nosso ambiente.

In [87]:
import numpy as np
import sympy as sp

Segue também algumas funções que ajudarão na implementação.

In [88]:
'''
    Descobre todas as variáveis livres de um sistema 

    Args:
        F: O sistema, representado como uma lista com todas as funções no formato simbólico do Sympy
    Retorno:
        Lista com todas as variaveis do sistema
'''
def variaveis(F: list[sp.Expr]) -> list[sp.Symbol]:
    allvars = [f.free_symbols for f in F]  # Cria uma lista com todas as variáveis de cada função
    # Aqui nós unimos todas as variáveis de todas as funções, tirando redundancia, depois ordenamos e transformamos numa lista
    return list(sorted(set.union(*allvars), key=str)) # type: ignore


'''
    Resolve numericamente o sistema
    Args:
        F: O sistema, representado como uma lista com todas as funções no formato simbólico do Sympy 
        X: O vetor solução
'''
def solve(F: list[sp.Expr], X: np.typing.NDArray) -> np.typing.NDArray:
    vars = variaveis(F) # Pega todas as variáveis do sistema

    if len(vars) < len(X): 
        print("Erro! Vetor solução possui mais entradas do que o sistema permite!")
        raise ArithmeticError
    if len(vars) > len(X):
        print("Erro! Vetor solução possui menos entradas do que o sistema precisa!")

    lF = [sp.lambdify(vars, f, "numpy") for f in F] # Transforma as funções do sistema numa função lambda
    return np.array([f(*X) for f in lF]) # Retorna os valores da solução


## Método de Newton-Raphson



### Matriz Jacobiana


Antes de explicarmos como funciona o método de Newton-Raphson, vamos precisar explicar o que é uma matriz Jacobiana. Uma matriz Jacobiana é uma matriz com todas as derivadas parciais de um sistema de equação. Seja o sistema $F=\{f_1,f_2,\dots,f_n\}$, em que cada função recebe as variáveis $\{x_1,x_2,\dots,x_n\}$:

$$
J_F(x_1,\dots,x_n)=\frac{\delta( f_1,\dots, f_n)}{\delta(x_1,\dots,x_n)}=
\begin{bmatrix}

\frac{\delta f_1}{\delta x_1} & \dots & \frac{\delta f_1}{\delta x_n} \\

\vdots & \ddots & \vdots \\

\frac{\delta f_m}{\delta x_1} & \dots & \frac{\delta f_m}{\delta x_n}

\end{bmatrix}
$$

Abaixo nós temos uma implementação da obtenção da matriz Jacobiana. Temos duas funções, uma para obter todas as variáveis da função, e outra para obter a matriz Jacobiana em questão. A implementação da função para obter variáveis foi feita para simplificar a utilização do algoritmo para o usuário

In [89]:
'''
    Retorna a Jacobiana de um sistema de funções.
    Args:
        F: O sistema, representado como uma lista com todas as funções no formato simbólico do Sympy
    Retorno:
        A matriz jacobiana, uma matriz com todas as derivadas parciais
'''
def jacobiana(F: list[sp.Expr]) -> list[list[sp.Expr]]:
    J = []
    derivadas = []

    vars = variaveis(F) # Pegar todas as variáveis encontradas no sistema

    for i in range(len(F)): # Começa a iterar pela lista de funções
        for j in vars: # Começa a iterar pela lista de variáveis
            derivadas.append(sp.diff(F[i], j)) # Adiciona numa lista a derivada parcial da função, para cada variável
        J.append(derivadas)
        derivadas = []

    return J # Transforma a matriz de derivadas parciais numa matriz do SymPy

### Método de Newton-Raphson



Agora o método em questão. Vamos precisar seguir um passo a passo, que será repetido a cada iteração.
Dados um sistema não linear $F$, uma estimativa inicial $x⁰$, $\epsilon_1$, $\epsilon_2 > 0$

1. Calcular $F(x^k)$ e $J(x^k)$
2. Se $\|F(x^k)\|_\infin<\epsilon_1$, então $x^*=x^k$ e pare; caso contrário, continue
3. Obtenha $S^k$, solução do sistema linear $J(x^k).S^k=-F(x^k)$
4. Faça: $x^{k+1}=x^k+S^k$
5. Se $\|x^{k+1}-x^k\|_\infin<\epsilon_2$, faça $x^*=x^{k+1}$ e pare; caso contrário, continue
6. $k=k+1$ e volte ao passo 1

Segue o algoritmo implementado em Python:

In [90]:
'''
    Descobrir os zeros das funções em um sistema não linear, aplicando o método de Newton-Raphson

    Args:
        F: O sistema não linear, representado por uma lista com as funções em formato simbólico do Sympy 
        estimativa: Uma lista com as estimativas iniciais
        epsilon1: Primeiro critério de parada (Fxk < epsilon)
        epsilon2: Segundo critério de parada (xk1 - xk < epsilon)
        maximoIteracoes: Maximo de iterações desejada [opcional]
        verbose: Verdadeiro caso queira descrição maior de como foi o ciclo de iterações
    
    Retorno:
        Um array numpy com os zeros das funções no sistema não linear F
'''
def newtonRaphson(F: list[sp.Expr], estimativa: np.typing.NDArray, epsilon1 = 10**-3, epsilon2 = 10**-3, maximoIteracoes=500, verbose=False) -> np.typing.NDArray[np.float64]:
    if len(F) != len(estimativa): # Verifica se o sistema tem tamanho diferente da estimativa inicial
        print("Tamanho da estimativa é diferente do sistema!")
        raise ArithmeticError # Levanta erro aritmético
    
    vars = variaveis(F) # Pega todas as variáveis do sistema de funções

    if len(estimativa) != len(vars): # Verifica se a quantidade de variáveis no sistema bate com a quantidade de valores na estimativa
        print("Estimativa inicial tem mais variáveis do que o sistema permite")
        raise ArithmeticError
    
    Fxk = np.array([])
    Jxk = np.array([])
    xk = np.array([np.float64(i) for i in estimativa]) # Transcrevendo a estimativa inicial para xk, para simplificar a leitura do algoritmo
    J = jacobiana(F) # Calcula a jacobiana da função
    if verbose:
        print(f'Variáveis encontradas: {vars}')
        print(f'Matriz Jacobiana: {J}')
    lF = [sp.lambdify(vars, f, "numpy") for f in F] # Transforma todos os símbolos em F para funções lambda
    lJ = [[sp.lambdify(vars, f, "numpy") for f in j] for j in J] # Transforma todos os símbolos em J para funções lambda

    # Começa as iterações
    for n in range(maximoIteracoes): # Critério de parada, para evitar infinitas iterações
        Fxk = np.array([f(*xk) for f in lF]) # Pegamos os resultados do sistema F
        Jxk = np.array([[f(*xk) for f in j] for j in lJ]) # Pegamos os resultados das funções da matriz Jacobiana

        if np.linalg.norm(Fxk, np.inf) < epsilon1: # Primeiro critério de parada, verifica se algum resultado Fxk é menor que o primeiro epsilon
            if verbose:
                print(f"Encontrado zero das funções no ciclo {n+1}: {sp.pprint(xk)}")
            return xk
        
        Sk = np.linalg.solve(Jxk, -Fxk) # Resolução do sistema linear
        xk1 = xk+Sk # Reescrevendo xk1
        if np.linalg.norm(xk1 - xk, np.inf) < epsilon2: # Segundo critério de parada, verifica se a diferença para o resultado anterior é menor que epsilon
            if verbose:
                print(f"Encontrado zero das funções no ciclo {n+1}: {xk}")
            return xk1
        
        if verbose:
            print(f"Matriz resposta no ciclo {n+1}: {xk}")
        xk = xk1 # Preparando para a próxima iteração

    print("Máximo de iterações ultrapassado!")
    if verbose:
        print("Matriz resposta encontrada: {xk}")
    return xk

## Solução de Sistemas

Agora nós temos tudo necessário para aplicar o método. Vamos testar com os seguintes sistemas:


### Sistema 1



$$
\begin{cases}

F_1(x_1,x_2)=\sin(x_1)+x_2^2-1=0 \\
F_2(x_1,x_2)=x_1^2+\cos(x_2)-2=0 \\

\end{cases}\\
x^0=\begin{pmatrix}
0,5\\0,5
\end{pmatrix} \\

\epsilon_1=\epsilon_2=10^{-3}

\\~\\\text{Resposta Esperada}\\~\\

x_4^*=\begin{pmatrix}
1,034\\0,375
\end{pmatrix}
$$

In [91]:
x1, x2 = sp.symbols('x1 x2')
print(newtonRaphson([sp.sin(x1)+x2**2-1, x1**2+sp.cos(x2)-2],np.array([0.5,0.5]), verbose=True))

Variáveis encontradas: [x1, x2]
Matriz Jacobiana: [[cos(x1), 2*x2], [2*x1, -sin(x2)]]
Matriz resposta no ciclo 1: [0.5 0.5]
Matriz resposta no ciclo 2: [1.20536546 0.15155803]
Matriz resposta no ciclo 3: [1.04312522 0.48488618]
Matriz resposta no ciclo 4: [1.03433425 0.38726511]
[1.03415167 0.37512454]
Encontrado zero das funções no ciclo 5: None
[1.03415167 0.37512454]


Ótimo! A resposta chegou no resultado esperado. Vamos testar com outro sistema

### Sistema 2

In [92]:
x1, x2 = sp.symbols('x1 x2')
F = [
    x1**2+x2**2-25,
    2*x1**2-x2
]
x0 = np.array([3.0,4.0])
newtonRaphson(F, x0)

array([1.54211646, 4.7562461 ])

### Sistema 3

In [93]:
x1, x2 = sp.symbols('x1 x2')
F = [
    x1**2+x2**2-2,
    sp.exp(x1-1)+x2**3-2
]
x0 = np.array([1.5,2.0])
print(newtonRaphson(F, x0, 10**-2,10**-2))

[0.99970847 1.00053483]


Maravilha, as respostas estão batendo! Vamos agora colocar em prova com o sistema que será avaliado.



### Sistema Avaliador



$$
x^0=\begin{pmatrix}0.1\\0.1\\-0.1\end{pmatrix}^t \\~\\

\begin{cases}
3x_1+\cos(x_2x_3)-\frac12=0\\
(x_1)^2-81(x_2+0.1)^2+\sin x_3+1.06 = 0\\
e^{-x_1x_2}+20x_3+\frac{10\pi-3}{3}=0
\end{cases}\\~\\


\text{Resposta esperada:}\\
\begin{pmatrix}0.5\\0\\-0.52359877\end{pmatrix}
$$

Não foi passado nenhum $\epsilon$, vamos chutar alguns valores até acertarmos. Vamos usar um lista de possíveis $\epsilon$, indo de $10^0$ até $10^-6$. Qualquer valor maior vai ser grande demais para um $\epsilon$, qualquer valor menor vai ajudar a chegar num valor mais preciso, mas não vai ser tão significativo a ponto de encontrar a resposta esperada. Para auxiliar a visualização, vamos criar uma função dedicada a isso.

In [ ]:
'''
     Chama a função Newton-Raphson diversas vezes, testando diferentes epsilons até chegarmos num valor próximo o suficiente

     Args:
          F: O sistema de funções
          X: O vetor de variáveis a ser testado
          esperado: Retorno esperado da chamada do Newton-Raphson
          epsilons: Lista com os possíveis epsilons a serem testados
          verbose: Se queremos com texto explicativo ou não
'''

def acharEpsilons(F: list[sp.Expr], X: np.typing.NDArray, esperado:np.typing.NDArray, epsilons: np.typing.NDArray, verbose=False) -> np.typing.NDArray:
     for i in epsilons:
          for j in epsilons:
               resp = newtonRaphson(F, X, epsilon1=i, epsilon2=j) # Chama a função Newton-Raphson com valores novos de epsilon
               if verbose:
                    print(f"Resposta para epsilon 1 = {i} e epsilon 2 = {j} \t{resp}")
               if np.allclose(esperado, resp): # Verifica se todos os valores obtidos possuem uma diferença de mais ou menos 10e-8
                    if verbose:
                         print(f"Valores de epsilon encontrados!\nEpsilon 1: {i}\nEpsilon 2: {j}")
                    return np.array([i,j]) # Se os valores forem próximos o suficiente, retornar os epsilons
     if verbose:
          print("Vetor solução não é válido!")
     return np.array([None,None]) # Se não achar, retornar epsilons vazios.


In [102]:

x1,x2,x3 = sp.symbols('x1 x2 x3')
x0 = np.array([0.1,0.1,-0.1])
F = [
     3*x1 + sp.cos(x2*x3) - 1/2,
     x1**2 - 81*(x2+0.1)**2 + sp.sin(x3) + 1.06,
     sp.exp(-x1*x2) + 20*x3 + (10*sp.pi-3)/3
]
esperado = np.array([0.5,0,-0.52359877])
epsilons = np.array([10e-1, 10e-2, 10e-3, 10e-4, 10e-5, 10e-6, 10e-7])


print(acharEpsilons(F, x0, esperado, epsilons))
print(newtonRaphson(F, x0))

[None None]
[-0.16665665 -0.01480728 -0.52347554]


Como podemos ver pela resposta, $\begin{pmatrix}0.5&0&-0.52359877\end{pmatrix}$ não é o vetor solução do sistema. O vetor solução do sistema é $\begin{pmatrix}-0.16665665&-0.01480728&-0.52347554\end{pmatrix}$. Para provar ainda mais, vamos passar ambas soluções como valores para as funções.

In [96]:
resultado = np.array([-0.16665665, -0.01480728, -0.52347554])
print(f"Resposta com valores obtidos:{solve(F, resultado)}")
print(f"Resposta com valores esperados:{solve(F, esperado)}")

Resposta com valores obtidos:[ 9.22070142e-09 -5.94876387e-07  2.26323108e-08]
Resposta com valores esperados:[2.00000000e+00 4.84826890e-09 1.11965978e-07]


Como podemos ver, o resultado obtido dá uma resposta muito mais próxima de $0$ para todas as funções. Vamos tentar modificar o sistema para ver se o vetor esperado funciona? Na resposta com os valores esperados, $x_2$ e $x_3$ são bem próximos de $0$. Próximos o suficiente para assumirmos que estão certos. Mas a resposta da primeira função dá $2$. E se subtrairmos $2$ da primeira função para termos 0?

In [97]:
F = [
     3*x1 + sp.cos(x2*x3) - 1/2 -2,
     x1**2 - 81*(x2+0.1)**2 + sp.sin(x3) + 1.06,
     sp.exp(-x1*x2) + 20*x3 + (10*sp.pi-3)/3
]

newtonRaphson(F, x0)

array([ 4.99999887e-01,  1.24118780e-05, -5.23598449e-01])

Podemos ver agora que modificando a primeira função nós temos uma resposta muito mais próxima do que esperado. Vamos testar com aquele verificador

In [98]:
acharEpsilons(F, x0, esperado, epsilons)

array([0.0001, 0.001 ])

### Conclusão

O método de Newton Raphson implementado funciona nos testes passados. O sistema avaliado não possui como resposta o vetor solução passado. Contudo, podemos modificar o sistema para vetor solução ser correto, dando uns dos possíveis epsilons $\epsilon_1 = 10^{-4}$ e $\epsilon_2 = 10^{-3}$.